This is a mockup of node2vec model created with stellar graph ml library

In [ ]:
import tensorflow as tf
import numpy as np
import networkx as nx

import stellar_ml as sml

Steps:
- Load a graph with GraphLoader
- Create an instance of random walk generator (class imported from the library); generate random walks on the graph, return random_walks
- Create an instance of (node,context) pairs generator (class imported from the library), connect to random_walks
- Build a tensorflow model for word2vec:
    - create placeholders for the input (node,context) pairs
    - create the layers: embeddings, context_softmax - use layers from the library, or layers from tf or keras?
    - create the loss:
            nce_loss = tf.reduce_mean(
                       tf.nn.nce_loss(weights=nce_weights,
                       biases=nce_biases,
                       labels=train_context,
                       inputs=embed,
                       num_sampled=num_sampled,
                       num_classes=vocabulary_size))
    - create an optimizer
- Train the word2vec model in a session
    - create minibatches of (node,context) pairs using the generator
    - loop through the minibatches, feeding them to the model
    - optimise the model's loss
    - repeat for n_epoch epochs
- Extract the final node embeddings

Load the graph:

In [ ]:
g = sml.graphloader.load('path/to/graph')  # this should support multiple graph formats: EPGM, networkx, graphml, and
                                            # the result should be a networkx graph (probably the most general MultiDiGraph)
    
assert hasattr(g, 'graph')   # G should be a networkx graph object

Create an instance of graph random walk generator, and use it to generate random walks on g:

In [ ]:
rw = sml.graph_exploration.UniformRandomWalk(g, numWalks, walkLength, p, q, seed)
walks = rw.generate()   # perform random walks on g and return a list of walks (node sequences)

In [ ]:
import networkx as nx
g = nx.Graph()